# Lab 3: Convolutional Neural Network

# Training a Classifier


**What about data?**


Generally, when you have to deal with image, text, audio or video data,
you can use standard python packages that load data into a numpy array.
Then you can convert this array into a ``torch.*Tensor``.

-  For images, packages such as Pillow, OpenCV are useful
-  For audio, packages such as scipy and librosa
-  For text, either raw Python or Cython based loading, or NLTK and
   SpaCy are useful

Specifically for vision, Pytorch team has created a package called
``torchvision``, that has data loaders for common datasets such as
Imagenet, CIFAR10, MNIST, etc. and data transformers for images, viz.,
``torchvision.datasets`` and ``torch.utils.data.DataLoader``.

This provides a huge convenience and avoids writing boilerplate code.

For this tutorial, we will use the CIFAR10 dataset.
It has the classes: ‘airplane’, ‘automobile’, ‘bird’, ‘cat’, ‘deer’,
‘dog’, ‘frog’, ‘horse’, ‘ship’, ‘truck’. The images in CIFAR-10 are of
size 3x32x32, i.e. 3-channel color images of 32x32 pixels in size.


   ![cifar10](https://miro.medium.com/max/1648/1*SZnidBt7CQ4Xqcag6rd8Ew.png)




# How to use Dataset and DataLoader?

## Data Loading

For convenience, PyTorch provides a number of utilities to load, preprocess and interact with datasets. These helper classes and functions are found in the torch.utils.data module. The two major concepts here are:



1.   A Dataset, which encapsulates a source of data,
2.   A DataLoader, which is responsible for loading a dataset, possibly in parallel.



New datasets are created by subclassing the torch.utils.data.Dataset class and overriding the \_\_len__ method to return the number of samples in the dataset and the \_\_getitem__ method to access a single value at a certain index.
​
For example, this would be a simple dataset encapsulating a range of integers:

In [ ]:
import math
import torch

class RangeDataset(torch.utils.data.Dataset):
  def __init__(self, start, end, step=1):
    self.start = start
    self.end = end
    self.step = step

  def __len__(self):
    return math.ceil((self.end - self.start) / self.step)

  def __getitem__(self, index):
    value = self.start + index * self.step
    assert value < self.end
    return value

Inside \_\_init__ we would usually configure some paths or change the set of samples ultimately returned.

In \_\_len__, we specify the upper bound for the index with which \_\_getitem__ may be called.

In \_\_getitem__ we return the actual sample, which could be an image or an audio snippet.

In [ ]:
dataset = RangeDataset(0, 10, 2)
print(len(dataset))

## DataLoader

To iterate over the dataset we could, in theory, simply have a for i in range loop and access samples via \_\_getitem__. However, it would be much more convenient if the dataset implemented the iterator protocol itself, so we could simply loop over samples with for sample in dataset. Fortunately, this functionality is provided by the DataLoader class.

A DataLoader object takes a dataset and a number of options that configure the way samples are retrieved. For example, it is possible to load samples in parallel, using multiple processes. For this, the DataLoader constructor takes a num_workers argument. Note that DataLoaders always return batches, whose size is set with the batch_size parameter. Here is a simple example:



In [ ]:
dataset = RangeDataset(0, 9)
data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=4,
    shuffle=True,  # set to True to have the data reshuffled at every epoch
    num_workers=0, # how many subprocesses to use for data loading
    drop_last=True # set to True to drop the last incomplete batch, if the dataset size is not divisible by the batch size.
  )

for i, batch in enumerate(data_loader):
  print(i, batch)

Use an iterator

See details [here](https://pytorch.org/docs/stable/data.html#torch.utils.data.IterableDataset)

In [ ]:
dataset_iter = iter(dataset)
print(next(dataset_iter))
print(next(dataset_iter))
print(next(dataset_iter))

In [ ]:
dataset_iter = iter(dataset)
for i in range(len(dataset)):
  element = next(iter(dataset_iter))
  print(element)

# Training an image classifier


We will do the following steps in order:

1. Load and normalizing the CIFAR10 training and test datasets using
   ``torchvision``
2. Define a Convolutional Neural Network
3. Define a loss function
4. Train the network on the training data
5. Test the network on the test data

1. Loading and normalizing CIFAR10


Using ``torchvision``, it’s extremely easy to load CIFAR10.




In [ ]:
%matplotlib inline

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

## 1- Load and normalizing the CIFAR10 training and test datasets using ``torchvision``

The output of torchvision datasets are PILImage images of range [0, 1].
We transform them to Tensors of normalized range [-1, 1].



In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 100

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
print('Size of a training batch: {}'.format(trainloader.batch_size))
print('Number of training batches: {}'.format(len(trainloader)))
print('Size of a test batch: {}'.format(testloader.batch_size))
print('Number of test batches: {}'.format(len(testloader)))

See the location of the downloaded data

In [ ]:
%pwd
%ls -l
%ls -l ./data
%ls -l ./data/cifar-10-batches-py/

Let us show some of the training images, for fun.



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

## 2- Define a Convolutional Neural Network


Copy the neural network from the Neural Networks section before and modify it to
take 3-channel images (instead of 1-channel images as it was defined).



The following neural network is created as a module. This is the best way to create a customized neural network (automatic differentiation is still working)

More information on nn.Module [here](https://pytorch.org/tutorials/beginner/nn_tutorial.html#refactor-using-nn-module)

### Question: code the class of the convolutional neural network.

The network must work as follows:

input -> convolution (kernel 5x5, depth 6) -> relu -> pooling 2D (2x2) ->

convolution (kernel 5x5, depth 16) -> relu -> pooling 2D (2x2) ->

fully connected layer with 120 neurons -> relu ->

fully connected layer with 84 neurons -> relu ->

fully connected layer with 10 neurons = output

You must replace the keywords "pass" by your code in the following cell.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        pass

    def forward(self, x):
        pass
        return x


net = Net()

## 3- Define a Loss function and optimizer


Let's use a Classification Cross-Entropy loss and SGD with momentum.



### Question: define the criterion (choose the appropriate one) and define the optimizer (SGD with an appropriate learning rate).


In [ ]:
import torch.optim as optim

criterion = ...
optimizer = ...

## 4- Train the network



This is when things start to get interesting.
We simply have to loop over our data iterator, and feed the inputs to the
network and optimize.



### Question: code the training of the network. Use the autograd mechanism and the optimizer.

Compute and print the running loss each time "Nprint" batches have been processed, i.e., compute and print the average of the loss over the "Nprint" mini-batches.

You must replace the keywords "pass" by your code in the following cell.

In [ ]:
Nprint = 100

nb_epoch = 2

for epoch in range(nb_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        pass

        # print statistics
        running_loss += loss.item()
        if i % Nprint == (Nprint-1):    # print every Nprint mini-batches
            pass

print('Finished Training')

## 5- Test the network on the test data



We have trained the network for 2 passes over the training dataset.
But we need to check if the network has learnt anything at all.

We will check this by predicting the class label that the neural network
outputs, and checking it against the ground-truth. If the prediction is
correct, we add the sample to the list of correct predictions.

Okay, first step. Let us display an image from the test set to get familiar.



In [ ]:
dataiter = iter(testloader)
images, labels = next(dataiter)

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

Okay, now let us see what the neural network thinks these examples above are:



In [ ]:
outputs = net(images)
print(outputs)

The outputs are energies for the 10 classes.
The higher the energy for a class, the more the network
thinks that the image is of the particular class.
So, let's get the index of the highest energy:



### Question: give the decision for all the images in the code cell above. The decision should be a label belonging to the object "classes".

In [ ]:
print(classes)

In [ ]:
# Write your code here.

Let us look at how the network performs on the whole dataset.



### Question: compute the average accuracy over all the test set.

In [ ]:
# Write your code here.

That looks way better than chance, which is 10% accuracy (randomly picking
a class out of 10 classes).
Seems like the network learnt something.

What are the classes that performed well, and the classes that did
not perform well




### Question: Compute the average accuracy per class (there are 10 classes).

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

# Write your code here.


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))